In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == torch.device('cpu'):
    print('Using cpu')
else:
    n_gpu = torch.cuda.device_count()
    print('Using {} GPUs'.format(torch.cuda.get_device_name(0)))

Using Tesla K80 GPUs


In [ ]:
! pip install datasets transformers seqeval

In [3]:
task = "ner"
batch_size = 8

# Загрузка данных - Primary Account Documents


In [4]:
!gdown --id 1cCXjHX9FAgouF0DWuWPO5qyBHVZDkXTQ

Downloading...
From: https://drive.google.com/uc?id=1cCXjHX9FAgouF0DWuWPO5qyBHVZDkXTQ
To: /content/NER_PAD_agg_test.csv
2.92MB [00:00, 93.8MB/s]


In [5]:
!gdown --id 1RFNBRcly96omdpNtYlEK6O1EuQs3ux7t

Downloading...
From: https://drive.google.com/uc?id=1RFNBRcly96omdpNtYlEK6O1EuQs3ux7t
To: /content/NER_PAD_agg_train.csv
11.7MB [00:00, 103MB/s]


In [6]:
!gdown --id 1AnLqXtSyJNBK7YmwuW6L6z2OYhE4RPTV

Downloading...
From: https://drive.google.com/uc?id=1AnLqXtSyJNBK7YmwuW6L6z2OYhE4RPTV
To: /content/NER_PAD_agg.csv
14.6MB [00:00, 88.5MB/s]


In [7]:
import pandas as pd
data = pd.read_csv('NER_PAD_agg.csv', converters={'tokens': eval, 'ner_tags': eval})

In [8]:
all_tags = set()
for _, row in data.iterrows():
    for tag in row.ner_tags:
        all_tags.add(tag)
all_tags = sorted(list(all_tags))

In [9]:
tag_to_idx = {t: i for i, t in enumerate(all_tags)}

In [10]:
label_list = {i: t for i, t in enumerate(all_tags)}

In [11]:
train = pd.read_csv('NER_PAD_agg_train.csv', converters={'tokens': eval, 'ner_tags': eval})
test = pd.read_csv('NER_PAD_agg_test.csv', converters={'tokens': eval, 'ner_tags': eval})

In [12]:
train.ner_tags = train.ner_tags.apply(lambda x : [tag_to_idx[label] for label in x])
test.ner_tags = test.ner_tags.apply(lambda x : [tag_to_idx[label] for label in x])

In [13]:
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')

In [14]:
from datasets import load_dataset, load_metric

In [15]:
datasets = load_dataset('pandas', data_files={'train' : 'train.pkl', 'test' : 'test.pkl'})

Using custom data configuration default-778450f13be47aa4


Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default-778450f13be47aa4/0.0.0/3547ff13f14ebf30dc8f11ab416dcf7721c18de2eb7e054bf9c80b88e765a791. Subsequent calls will reuse this data.


In [16]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3200
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 800
    })
})

# Токенизатор

In [17]:
from transformers import AutoTokenizer
model_checkpoint = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [18]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [19]:
label_all_tokens = False

In [20]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [21]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Warning : есть последовательности больше, чем максимально возможная в BERT - 512

In [22]:
pd.Series(tokenized_datasets["train"]['input_ids']).apply(lambda x : len(x)).describe()

count    3200.000000
mean      348.981875
std       236.542223
min         6.000000
25%       151.750000
50%       308.000000
75%       434.000000
max      4288.000000
dtype: float64

In [23]:
pd.Series(tokenized_datasets["test"]['input_ids']).apply(lambda x : len(x)).describe()

count     800.000000
mean      347.810000
std       230.390747
min        36.000000
25%       145.750000
50%       307.500000
75%       431.750000
max      1471.000000
dtype: float64

In [24]:
train_tokenized = tokenized_datasets["train"].to_pandas()
test_tokenized = tokenized_datasets["test"].to_pandas()

In [25]:
train_tokenized.head(1)

,attention_mask,id,input_ids,labels,ner_tags,token_type_ids,tokens
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",SV990115834_ОРГ__INM_16'04'09_3.pdf,"[101, 254, 3274, 874, 144, 130, 274, 94901, 11...","[-100, 15, 15, -100, 15, -100, 15, 15, -100, -...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[l, испр, 4-, v, moscow, quarterly, report, li..."


In [26]:
test_tokenized.head(1)

,attention_mask,id,input_ids,labels,ner_tags,token_type_ids,tokens
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",B31005082_ОРГ__INM_15'39'47_1.pdf,"[101, 254, 867, 106, 32877, 905, 138, 34193, 1...","[-100, 15, 15, 15, 15, 15, 15, 15, 15, 15, -10...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[l, т, !, приложение, №, 1, постановлению, пра..."


In [27]:
CHUNCK_SIZE = 512

def lists_to_chuncks(row):
    n = CHUNCK_SIZE - 2
    new_row = row.copy()
    for key in ['attention_mask', 'input_ids', 'labels', 'token_type_ids']:
        FIRST = new_row[key][0]
        LAST = new_row[key][-1]
        new_row[key] = new_row[key][1:-1]
        new_row[key] = [[FIRST] + list(new_row[key][i:i + n]) + [LAST] for i in range(0, len(new_row[key]), n)]
    return new_row

In [28]:
train_tokenized = train_tokenized.drop(columns = ['ner_tags', 'tokens']).apply(lists_to_chuncks, axis=1)
train_tokenized = train_tokenized.set_index(['id']).apply(pd.Series.explode).reset_index()

In [29]:
test_tokenized = test_tokenized.drop(columns = ['ner_tags', 'tokens']).apply(lists_to_chuncks, axis=1)
test_tokenized = test_tokenized.set_index(['id']).apply(pd.Series.explode).reset_index()

In [30]:
train_tokenized.to_pickle('train_tokenized.pkl')
test_tokenized.to_pickle('test_tokenized.pkl')

In [31]:
tokenized_datasets = load_dataset('pandas', data_files={'train' : 'train_tokenized.pkl', 'test' : 'test_tokenized.pkl'})

Using custom data configuration default-82b64f2d0bc3be20


Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default-82b64f2d0bc3be20/0.0.0/3547ff13f14ebf30dc8f11ab416dcf7721c18de2eb7e054bf9c80b88e765a791. Subsequent calls will reuse this data.


In [32]:
pd.Series(tokenized_datasets["train"]['input_ids']).apply(lambda x : len(x)).describe()

count    3778.000000
mean      295.896771
std       145.507633
min         4.000000
25%       145.250000
50%       294.000000
75%       421.000000
max       512.000000
dtype: float64

In [33]:
pd.Series(tokenized_datasets["test"]['input_ids']).apply(lambda x : len(x)).describe()

count    943.000000
mean     295.370095
std      144.487497
min        3.000000
25%      143.500000
50%      295.000000
75%      419.500000
max      512.000000
dtype: float64

Теперь все последовательности нужной длины

# Модель


# Обучение

In [34]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Warning : модель надо обучить на наших данных

In [35]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [36]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [37]:
metric = load_metric("seqeval")

In [38]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [39]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [40]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.055935,0.791966,0.733578,0.761655,0.984060
2,0.128500,0.041912,0.829713,0.824759,0.827229,0.988024
3,0.047500,0.041513,0.822467,0.857602,0.839667,0.988232


TrainOutput(global_step=1419, training_loss=0.07203426992834412, metrics={'train_runtime': 2498.6232, 'train_samples_per_second': 0.568, 'total_flos': 1732874538208512.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 741257216, 'init_mem_gpu_alloc_delta': 709382656, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 87965696, 'train_mem_gpu_alloc_delta': 2197097472, 'train_mem_cpu_peaked_delta': 325816320, 'train_mem_gpu_peaked_delta': 6528570368})

# Сохранение модели


In [41]:
import pickle
with open("PAD_BERT[DeepPavlov_rubert_base_cased].pkl", 'wb') as f:
    pickle.dump(model, f)

# Валидация

In [42]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 0.9882317942189813,
 'eval_f1': 0.8396671913649651,
 'eval_loss': 0.0415133461356163,
 'eval_mem_cpu_alloc_delta': 3104768,
 'eval_mem_cpu_peaked_delta': 1921024,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 496390656,
 'eval_precision': 0.822466960352423,
 'eval_recall': 0.8576022048690859,
 'eval_runtime': 68.189,
 'eval_samples_per_second': 13.829}

In [43]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'DOCAGRDATE': {'f1': 0.7319884726224785,
  'number': 150,
  'precision': 0.6446700507614214,
  'recall': 0.8466666666666667},
 'DOCAGRNUM': {'f1': 0.5988700564971752,
  'number': 179,
  'precision': 0.6057142857142858,
  'recall': 0.5921787709497207},
 'DOCAMOUNT': {'f1': 0.8474970896391153,
  'number': 1243,
  'precision': 0.8185907046476761,
  'recall': 0.8785197103781175},
 'DOCCPTY': {'f1': 0.8407148407148408,
  'number': 595,
  'precision': 0.7817919075144508,
  'recall': 0.9092436974789916},
 'DOCCPTYINN': {'f1': 0.9439728353140917,
  'number': 282,
  'precision': 0.9055374592833876,
  'recall': 0.9858156028368794},
 'DOCCUSTOMER': {'f1': 0.9355877616747181,
  'number': 609,
  'precision': 0.9178515007898894,
  'recall': 0.9540229885057471},
 'DOCDATE': {'f1': 0.8742603550295858,
  'number': 645,
  'precision': 0.8359264497878359,
  'recall': 0.9162790697674419},
 'DOCNUM': {'f1': 0.7294938917975567,
  'number': 651,
  'precision': 0.8444444444444444,
  'recall': 0.6420890937019

In [44]:
pd.DataFrame.from_dict(results).transpose().iloc[:-4, :-1]

,precision,recall,f1
DOCAGRDATE,0.644670,0.846667,0.731988
DOCAGRNUM,0.605714,0.592179,0.598870
DOCAMOUNT,0.818591,0.878520,0.847497
DOCCPTY,0.781792,0.909244,0.840715
DOCCPTYINN,0.905537,0.985816,0.943973
DOCCUSTOMER,0.917852,0.954023,0.935588
DOCDATE,0.835926,0.916279,0.874260
DOCNUM,0.844444,0.642089,0.729494


In [45]:
pd.DataFrame.from_dict(results).transpose().iloc[-4:, 0].rename('result')

overall_precision    0.822467
overall_recall       0.857602
overall_f1           0.839667
overall_accuracy     0.988232
Name: result, dtype: float64

In [46]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 2.8MB/s 


In [48]:
print(metrics.flat_classification_report(true_labels, true_predictions, labels=all_tags[:-1], digits=3))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

 B-DOCAGRDATE      0.695     0.867     0.772       150
  B-DOCAGRNUM      0.640     0.626     0.633       179
  B-DOCAMOUNT      0.844     0.887     0.865      1243
    B-DOCCPTY      0.829     0.923     0.874       595
 B-DOCCPTYINN      0.906     0.986     0.944       282
B-DOCCUSTOMER      0.931     0.952     0.942       609
    B-DOCDATE      0.874     0.936     0.904       645
     B-DOCNUM      0.844     0.642     0.729       651
 I-DOCAGRDATE      0.652     0.830     0.730        88
  I-DOCAGRNUM      0.000     0.000     0.000         7
  I-DOCAMOUNT      0.796     0.877     0.835       620
    I-DOCCPTY      0.822     0.933     0.874       564
I-DOCCUSTOMER      0.916     0.948     0.932       462
    I-DOCDATE      0.929     0.957     0.943       605
     I-DOCNUM      0.000     0.000     0.000         0

    micro avg      0.850     0.886     0.867      6700
    macro avg      0.712     0.758     0.732      6700
 weighte